## Sanity checks for evaluating AIP execution on the RGP CAGI sample set.

In [1]:
import pandas as pd
import re
import itertools

from reanalysis.utils import read_json_from_path
from cpg_utils import to_path

from peddy import Ped

In [2]:
# Convenience functions.

def hail_az_to_https(hail_az: str) -> str:
    """converts a hail-az prefixed Azure storage path to a standard https prefixed path."""
    return re.sub(r"^hail-az://(?P<container>.*?)/", "https://\g<container>.blob.core.windows.net/", hail_az)

def https_to_hail_az(https: str) -> str:
    """converts a https prefixed Azure storage path to a hail-az prefixed path."""
    return re.sub(r"^https://(?P<container>.*?).blob.core.windows.net/", "hail-az://\g<container>/", https)

def family_from_sid(sid: str) -> str:
    """Converts and RGP sample_id to the corresponding family_id."""
    return '_'.join(sid.split('_')[:2])


In [3]:
# Relevant paths, https formatted.

AIP_RESULTS_PATH = 'https://raregen001sa.blob.core.windows.net/test-analysis/reanalysis_train/2023-01-31/summary_results.json'
COMPARISON_RESULTS_PATH = 'https://raregen001sa.blob.core.windows.net/test/reanalysis_train/comparison/2023-02-01/comparison_result_match_summary.json'
ANSWER_KEY_PATH = 'https://raregen001sa.blob.core.windows.net/test/inputs/rgp/CAGI6_RGP Training Set Key.xlsx'
PEDIGREE_PATH = 'https://raregen001sa.blob.core.windows.net/test/inputs/rgp/rgp_train.fam'


In [12]:
# Load and format the input pedigree.
with open(to_path(PEDIGREE_PATH), 'r') as f:
    pedigree = Ped(f)

# Load and format the answer key.
with open(to_path(ANSWER_KEY_PATH), 'rb') as f:
    answer_key = pd.read_excel(f, dtype=str)

answer_key = answer_key.rename(columns={'subject_id (to delete before sharing)': 'sample_id'})
answer_key['family_id'] = answer_key.apply(lambda x: family_from_sid(x['sample_id']), axis=1)

# Load and format the AIP results.
aip_results = read_json_from_path(AIP_RESULTS_PATH)

# Load and format the comparison results.
comp_results = read_json_from_path(COMPARISON_RESULTS_PATH)


EnvironmentCredential.get_token failed: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource.
ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource.
EnvironmentCredential.get_token failed: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource.
ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource.
EnvironmentCredential.get_token failed: EnvironmentCredential authentication unavailable. Environment 

In [33]:
from comparison.comparison import find_affected_samples, prep_rgp_dataframe, common_format_dataframe



Package                           Version   Editable project location
--------------------------------- --------- -------------------------------------------------
aiohttp                           3.8.1
aiohttp-session                   2.7.0
aiosignal                         1.2.0
airtable-python-wrapper           0.15.3
analysis-runner-ms                0.9.9
Arpeggio                          1.10.2
asttokens                         2.0.8
async-timeout                     4.0.2
asyncinit                         0.2.4
attrs                             22.1.0
automated-interpretation-pipeline 0.1.0     /home/azureuser/automated-interpretation-pipeline
avro                              1.11.1
azure-common                      1.1.28
azure-core                        1.26.0
azure-identity                    1.6.0
azure-keyvault-secrets            4.6.0
azure-mgmt-core                   1.3.2
azure-mgmt-storage                20.1.0
azure-storage-blob                12.11.0
backcall     

In [5]:
# Sanity check 1 - the same set of families is represented in all the files under comparison
aip_families = set([family_from_sid(sid) for sid in aip_results['results'].keys()])
key_families = set(answer_key['family_id'])
pedigree_families = set(pedigree.families)

print('Here is a list of all discrepancies between the various family_id lists:')
any_found = False
for a_name, b_name in itertools.permutations(['aip_families', 'key_families', 'pedigree_families'], 2):
    a = locals()[a_name]
    b = locals()[b_name]
    if diff := a.difference(b):
        print(f'  {a_name}-{b_name} = {diff}')
        any_found = True
if not any_found:
    print('  None found')

Here is a list of all discrepancies between the various family_id lists:
  key_families-aip_families = {'RGP_74'}
  pedigree_families-aip_families = {'RGP_74'}


In [6]:
# Cohort summary statistics.

print(f'Total number of families: {len(pedigree.families)}')
print(f'Number of samples analyzed: {len(list(pedigree.samples()))}')
print(f'Total number of families with solves: {len(key_families)}')

Total number of families: 35
Number of samples analyzed: 90
Total number of families with solves: 35


In [7]:
# seqr labels compared with the "answer key".

print(f'Analyses included the following labels from SEQR: {comp_results.keys()}')
label_dist = {k: comp_results[k]['matched']['count'] + comp_results[k]['unmatched']['count'] for k in comp_results if comp_results[k]['matched']['count'] + comp_results[k]['unmatched']['count'] > 0}
print(f'The distribution of labeled variants in this set was: {label_dist}')





Analyses included the following labels from SEQR: dict_keys(['KGFP', 'T1_NGAP', 'T1_NGFKP', 'T1_PE', 'T1_NMOI', 'T1_KGNP', 'T2_NGAP', 'T2_NGFKP', 'T2_PE', 'T2_PND', 'T2_KGNP'])
The distribution of labeled variants in this set was: {'KGFP': 36, 'T1_NGAP': 3, 'T1_PE': 2}


In [8]:
data = []

for label in comp_results:
    for status in comp_results[label]:
        for variant in comp_results[label][status]['details']:
            m = re.match('(?P<sid>RGP_[0-9]+_[0-9]+)::(?P<var>[0-9X]+:[0-9]+_[A-Z]+>[A-Z]+) - Confidence.(?P<label>[0-9A-Za-z_]+)', variant)
            if not m:
                print(f'    UNMATCHED: {variant}')

            assert m.group('label') == label
            data.append([m.group('sid'), family_from_sid(m.group('sid')), m.group('var'), status, label])

comp_results_df = pd.DataFrame(data, columns=['sample_id', 'family_id', 'variant_raw', 'match_status', 'seqr_label'])

# TODO set reasonable index.


In [24]:
from comparison.comparison import CommonFormatResult, Confidence
from collections import defaultdict
import math

sample_dict = defaultdict(list)

for _, row in answer_key.iterrows():
    # There's probably a slicker way to do this, but this is easy to reason about and debug.

    # Check whether this row is a proband.
    if not row['CAGI-RGP ID'].endswith('PROBAND'):
        continue

    # Check for a compound het (a second variant), if found, treat it just like a primary variant (for now)
    if isinstance(row['Gene-2'], str):
        support_result = CommonFormatResult(row['Chrom-2'], int(row['Pos-2']), row['Ref-2'], row['Alt-2'], confidence=[Confidence.EXPECTED])
        #sample_dict[row['sample_id']].append(support_result)
    else:
        support_result = None

    sample_dict[row['sample_id']].append(
        CommonFormatResult(row['Chrom-1'], int(row['Pos-1']), row['Ref-1'], row['Alt-1'], confidence=[Confidence.EXPECTED], support=support_result)
    )



RGP_1105_3 : [1:155612073_C>G (1:155612209_G>C)- Confidence.EXPECTED]
RGP_1133_3 : [18:23535687_A>G (18:23536811_G>A)- Confidence.EXPECTED]
RGP_1173_3 : [2:240783791_A>G - Confidence.EXPECTED]
RGP_1197_3 : [16:57901371_T>A - Confidence.EXPECTED]
RGP_1203_3 : [15:45069018_CG>C - Confidence.EXPECTED]
RGP_183_3 : [7:152222037_GT>G - Confidence.EXPECTED]
RGP_19_3 : [17:38739601_G>A - Confidence.EXPECTED]
RGP_218_3 : [3:132675903_G>A (3:132675248_G>C)- Confidence.EXPECTED]
RGP_320_3 : [X:154031020_G>A - Confidence.EXPECTED]
RGP_362_3 : [18:51078294_C>T - Confidence.EXPECTED]
RGP_405_3 : [2:47808215_GAC>G - Confidence.EXPECTED]
RGP_409_3 : [X:101412604_C>T - Confidence.EXPECTED]
RGP_416_3 : [5:177267613_G>T - Confidence.EXPECTED]
RGP_426_3 : [22:41152220_CTG>C - Confidence.EXPECTED]
RGP_437_3 : [6:157200782_C>G - Confidence.EXPECTED]
RGP_440_3 : [1:1806503_A>G - Confidence.EXPECTED]
RGP_441_3 : [9:127676671_T>C - Confidence.EXPECTED]
RGP_476_3 : [11:17772388_G>A - Confidence.EXPECTED]
RGP_47

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no


In [20]:
None == None

True

In [28]:
aip_results['results']

{'RGP_1105_3': {'variants': [{'sample': 'RGP_1105_3',
    'family': 'RGP_1105',
    'gene': 'ENSG00000125459',
    'var_data': {'coords': {'chrom': '1',
      'pos': 155612209,
      'ref': 'G',
      'alt': 'C'},
     'info': {'ac': 2,
      'af': 0.000678399985190481,
      'an': 2948,
      'as_inbreedingcoeff': -0.000699999975040555,
      'as_qd': 10.15999984741211,
      'baseqranksum': 3.75,
      'dp': 48222,
      'excesshet': 3.0118000507354736,
      'fs': 5.270999908447266,
      'inbreedingcoeff': -0.000699999975040555,
      'mleac': 2,
      'mleaf': 0.000678399985190481,
      'mq': 56.869998931884766,
      'mqranksum': 0.42899999022483826,
      'mq_dp': 96,
      'negative_train_site': True,
      'qd': 10.15999984741211,
      'raw_mq': 310456.0,
      'readposranksum': 1.0099999904632568,
      'sor': 1.2339999675750732,
      'vqslod': -1.3359999656677246,
      'culprit': 'MQ',
      'clinvar_sig': 'Pathogenic',
      'clinvar_stars': 1,
      'clinvar_allele': 9